In [1]:
import config
import geopandas as gpd
import json
import networkx as nx
import numpy as np
import os
import osmnx as ox
import pandas as pd
import time

ox.config(use_cache=True,
          log_file=True,
          log_console=True,
          log_filename='calculate-urbanized_areas',
          cache_folder=config.urbanized_areas_cache_folder)

print(ox.__version__)
print(nx.__version__)

0.8.1
2.1


In [2]:
graphml_folder = config.urbanized_areas_graphml_folder
uas_folder = 'input_data/urbanized_areas' #tiger urban areas shapefiles
stats_folder = config.urbanized_areas_stats_folder

In [3]:
urbanized_areas = []
for state_folder in os.listdir(graphml_folder):
    for urbanized_area_file in os.listdir('{}/{}'.format(graphml_folder, state_folder)):

        data = {}
        data['state_folder'] = state_folder
        data['state_fips'] = state_folder.split('_')[0]
        data['state'] = state_folder.split('_')[1]
        data['urbanized_area_file'] = urbanized_area_file
        data['geoid'] = urbanized_area_file.split('_')[0]
        data['urbanized_area'] = urbanized_area_file.strip('_{}'.format(data['geoid'])).replace('.graphml', '').replace('_', ' ')
        urbanized_areas.append(data)

df = pd.DataFrame(urbanized_areas)
len(df)

497

## Load the UAs shapefile

In [4]:
gdf = gpd.read_file(uas_folder).sort_values(by='ALAND10', ascending=True)
gdf = gdf[~gdf['NAMELSAD10'].str.contains('Urban Cluster')]
gdf = gdf[['GEOID10', 'ALAND10']]
len(gdf)

497

In [5]:
df = pd.merge(df, gdf, how='left', left_on='geoid', right_on='GEOID10').drop(columns=['GEOID10'])

## Calculate the stats

In [6]:
def load_graph_get_stats(row):
    
    start_time = time.time()
    name = row['urbanized_area']
    geoid = row['geoid']
    land_area = row['ALAND10']
    
    
    
    try:
        folder = '{}/{}'.format(graphml_folder, row['state_folder'])
        filename = row['urbanized_area_file']
        G = ox.load_graphml(folder=folder, filename=filename)
        
        stats = ox.basic_stats(G, area=land_area)
        stats['name'] = name
        stats['geoid'] = geoid
        stats['area'] = land_area
        
        # unpack k-counts and k-proportion dicts into individiual keys:values
        for k, count in stats['streets_per_node_counts'].items():
            stats['int_{}_streets_count'.format(k)] = count
        for k, proportion in stats['streets_per_node_proportion'].items():
            stats['int_{}_streets_prop'.format(k)] = proportion
            
        # calculate/drop the extended stats that have values per node
        extended_stats = ox.extended_stats(G)
        se = pd.Series(extended_stats)
        se = se.drop(['avg_neighbor_degree', 'avg_weighted_neighbor_degree', 'clustering_coefficient',
                      'clustering_coefficient_weighted', 'degree_centrality', 'pagerank'])
        extended_stats_clean = se.to_dict()
        
        for key in extended_stats_clean:
            stats[key] = extended_stats_clean[key]
        
        stats['area_km'] = land_area / 1e6
        stats['time'] = time.time()-start_time
        return pd.Series(stats)

    except Exception as e:
        print('{} failed: {}'.format(name, e))

In [7]:
stats_temp = df.apply(load_graph_get_stats, axis=1)
stats_temp.shape

(497, 66)

In [8]:
stats_temp['time'].sum()

5578.007314920425

## Clean up the dataframe

In [9]:
stats = stats_temp.copy()

In [10]:
# stuff to drop
cols_to_drop = ['area', 'time', 'streets_per_node_counts', 'streets_per_node_proportion', 
                'pagerank_max_node', 'pagerank_min_node', 'clean_intersection_count',
                'clean_intersection_density_km']

In [11]:
cols_to_rename = {}
for col in stats.columns:
    if 'int_' in col:
        n = col.split('_')[1]
        if n not in ['1', '3', '4']:
            cols_to_drop.append(col)
        else:
            suffix = 'count' if 'count' in col else 'proportion'
            cols_to_rename[col] = 'intersect_{}way_{}'.format(n, suffix)
            
stats = stats.drop(cols_to_drop, axis=1)

In [12]:
# rename these to friendlier names
cols_to_rename['clustering_coefficient_avg'] = 'cluster_coeff_avg'
cols_to_rename['clustering_coefficient_weighted_avg'] = 'cluster_coeff_weighted_avg'
cols_to_rename['intersection_density_km'] = 'intersect_density_km'
cols_to_rename['intersect_1way_count'] = 'dead_end_count'
cols_to_rename['intersect_1way_proportion'] = 'dead_end_proportion'
cols_to_rename['m'] = 'edge_count'
cols_to_rename['n'] = 'node_count'
cols_to_rename['name'] = 'urbanized_area'
stats = stats.rename(columns=cols_to_rename)
stats = stats.rename(columns=cols_to_rename)

In [13]:
# drop anything lacking a GEOID
stats = stats.dropna(subset=['geoid'])

In [14]:
# make these integers
cols_int = ['intersection_count', 'edge_length_total', 'edge_count', 'node_count', 'street_segments_count']
stats[cols_int] = stats[cols_int].astype(np.int64)

In [15]:
# make city, state, geoid at left of df
cols = stats.columns.tolist()
cols.insert(0, cols.pop(cols.index('urbanized_area')))
cols.insert(1, cols.pop(cols.index('geoid')))
stats = stats.reindex(columns=cols)

## View the results

In [16]:
stats.shape

(497, 32)

In [17]:
stats.columns

Index(['urbanized_area', 'geoid', 'area_km', 'avg_neighbor_degree_avg',
       'avg_weighted_neighbor_degree_avg', 'circuity_avg', 'cluster_coeff_avg',
       'cluster_coeff_weighted_avg', 'degree_centrality_avg',
       'edge_density_km', 'edge_length_avg', 'edge_length_total',
       'dead_end_count', 'dead_end_proportion', 'intersect_3way_count',
       'intersect_3way_proportion', 'intersect_4way_count',
       'intersect_4way_proportion', 'intersection_count',
       'intersect_density_km', 'k_avg', 'edge_count', 'node_count',
       'node_density_km', 'pagerank_max', 'pagerank_min',
       'self_loop_proportion', 'street_density_km', 'street_length_avg',
       'street_length_total', 'street_segments_count', 'streets_per_node_avg'],
      dtype='object')

In [18]:
stats.sort_values('area_km').tail()

,urbanized_area,geoid,area_km,avg_neighbor_degree_avg,avg_weighted_neighbor_degree_avg,circuity_avg,cluster_coeff_avg,cluster_coeff_weighted_avg,degree_centrality_avg,edge_density_km,...,node_count,node_density_km,pagerank_max,pagerank_min,self_loop_proportion,street_density_km,street_length_avg,street_length_total,street_segments_count,streets_per_node_avg
225,Boston MA--NH--RI Urbanized Area,09271,4852.213624,2.758296,0.030111,1.089640,0.051125,0.000592,0.000034,11624.279212,...,146740,30.241867,0.000026,1.026301e-06,0.023993,6381.980932,153.694796,3.096673e+07,201482,2.708171
364,Philadelphia PA--NJ--DE--MD Urbanized Area,69076,5131.722319,2.805306,0.049110,1.081258,0.049180,0.000385,0.000031,13126.376752,...,164592,32.073442,0.000028,9.154048e-07,0.009922,7296.163610,158.556655,3.744189e+07,236142,2.871215
162,Chicago IL--IN Urbanized Area,16264,6323.669758,2.941123,0.028676,1.066183,0.036999,0.000655,0.000026,14156.597794,...,209734,33.166501,0.000017,7.181088e-07,0.010919,7837.721468,161.371512,4.956316e+07,307137,2.918173
136,Atlanta GA Urbanized Area,03817,6850.603579,2.733105,0.028639,1.099946,0.041925,0.000516,0.000024,11582.767638,...,195872,28.591933,0.000021,7.685879e-07,0.010679,6257.247296,177.171438,4.286592e+07,241946,2.466085
297,New York--Newark NY--NJ--CT Urbanized Area,63217,8934.395320,2.858386,0.033349,1.060921,0.039564,0.000264,0.000014,15764.652896,...,374570,41.924494,0.000019,4.037549e-07,0.006985,8840.077138,147.381830,7.898074e+07,535892,2.857533


## Save to disk

In [19]:
if not os.path.exists(stats_folder):
    os.makedirs(stats_folder)
output_path = '{}/urbanized_areas-stats.csv'.format(stats_folder)
stats.to_csv(output_path, encoding='utf-8', index=False)